In [ ]:
# Mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import os
import glob
import os.path
import datetime

In [ ]:
# Uncomment out if you want to see all of the actual file names by .plt file
# for dirname, _, filenames in os.walk('/content/drive/MyDrive/Berkley/DATASCI209/'):
    # for filename in filenames:
        # print(os.path.join(dirname, filename))

In [ ]:
mode_names = ['walk', 'bike', 'bus', 'car', 'subway','train', 'airplane', 'boat', 'run', 'motorcycle', 'taxi']
mode_ids = {s : i + 1 for i, s in enumerate(mode_names)}

In [ ]:
mode_ids

{'walk': 1,
 'bike': 2,
 'bus': 3,
 'car': 4,
 'subway': 5,
 'train': 6,
 'airplane': 7,
 'boat': 8,
 'run': 9,
 'motorcycle': 10,
 'taxi': 11}

In [ ]:
# Create three different functions to clean up the data into something that is usable

# 1 - Create a function to read the .plt files & creates two new columns (day, hour) from the time column
def read_plt(plt_file):

    points = pd.read_csv(plt_file, skiprows=6, header=None)

    # for clarity rename columns
    points.rename(inplace=True, columns={0: 'lat', 1: 'lon', 3: 'alt', 5: 'day', 6: 'hour'})

    date_format = '%Y-%m-%d %H:%M:%S'
    points['time'] = pd.to_datetime(points['day'] + ' ' + points['hour'], format=date_format)

    # remove unused columns
    points.drop(inplace=True, columns=[2, 4, 'day', 'hour'])

    # # add 8 hours to the time column and change the date if needed
    # points['time'] = points['time'] + datetime.timedelta(hours=8)

    # points['year'] = points['time'].dt.year
    # points['day_of_week'] = points['time'].dt.day_name()
    # points['month'] = points['time'].dt.month_name()
    # points['hour'] = points['time'].dt.hour

    return points

# 2 - Create a function that read the labels from the .txt file
def read_labels(labels_file):
    labels = pd.read_csv(labels_file, skiprows=1, header=None, sep='\s+')
    labels['start_time'] = pd.to_datetime(labels[0] + ' ' + labels[1])
    labels['end_time'] = pd.to_datetime(labels[2] + ' ' + labels[3])
    labels.rename(inplace=True, columns = {4: 'label'})
    labels.drop(inplace=True, columns=[0,1,2,3])
    labels['label'] = [mode_ids[i] for i in labels['label']]
    labels['segment'] = labels.index

    return labels

# 3 - Create a function that applies each of the labels to the points
def apply_labels(points, labels):
    indices = labels['start_time'].searchsorted(points['time'], side='right') - 1
    no_label_condition = (indices < 0) | (points['time'].values > labels['end_time'].iloc[indices].values)

    points['label'] = labels['label'].iloc[indices].values
    points['segment'] = labels['segment'].iloc[indices].values

    points.loc[no_label_condition, 'label'] = 0
    points.loc[no_label_condition, 'segment'] = 0

    # Print the points row that has a 0 in the label
    # print(points[points['label'] == 0].head())

    print('total points', points.shape[0])
    print('total points not linked to the label', no_label_condition.sum())
    print('percent of points with no label', no_label_condition.sum() / points.shape[0]*100)

    results = [points.shape[0], no_label_condition.sum(), no_label_condition.sum() / points.shape[0]*100]

    # Drop the rows that have 0 in label and segment
    points.drop(points[points['label'] == 0].index, inplace=True)
    points.drop(points[points['segment'] == 0].index, inplace=True)

    # Add a new column that indicates start, move or stop based on if it is the first row for that user for that segment
    points['start_go_stop'] = np.where(points['segment'] == points['segment'].shift(1), 'go', 'start')
    points['start_go_stop'] = np.where(points['segment'] == points['segment'].shift(-1), points['start_go_stop'], 'stop')

    # print('total points without a label', points[points['label'] == 0].shape[0])
    # print('total points without a segment', points[points['segment'] == 0].shape[0])

    return points, results

# 4 - Create a function that reads all the users in the folder
def read_all_users_2(folder):

    # Get all subfolders of your folder
    subfolders = [sf for sf in os.listdir(folder)
              if os.path.isdir(os.path.join(folder, sf))]

    # subfolders = os.listdir(folder)

    # Create an empty list for the compiled list of activities
    dfs = []

    # Create an empty list for the compiled list of matching results
    results = []

    # Go through each of the subfolders using i for the iteration number,
    # and sf for the subfolder name
    for i, sf in enumerate(subfolders):

      # Go through each of the objects in that subfolder
      for j in os.listdir(os.path.join(folder, sf)):

        # Only select subfolder/user that have a label file, i.e. a text file
        if j.endswith('.txt'):

          # For those subfolder/users, read the labels file using the
          # read_labels function
          labels = read_labels(os.path.join(folder, sf, j))

          # For those subfolders/users, read all the plot files and
          # concatonate them into one singular dataframe
          plt_files = glob.glob(os.path.join(folder, sf, 'Trajectory', '*.plt'))
          df = pd.concat([read_plt(f) for f in plt_files])
          df.reset_index(inplace=True, drop=True)
          # order the data based off of the 'time' column
          df.sort_values(by='time', inplace=True)

          # Add a new column to the dataframe to identify the subfolder/user
          # for that activity
          df['user'] = int(sf)

          print('user', sf)

          # Apply the labels to the dataframe
          df, match_results = apply_labels(df, labels)

          # Append the overall dataframe with the new dataframe
          # for that specific subfolder/users
          dfs.append(df)

          # Add the user # to the match_results
          match_results.insert(0, sf)

          # # append the match results to the results list
          results.append(match_results)

        else:
          pass

    return pd.concat(dfs), results

In [ ]:
# plt_file = "/content/drive/MyDrive/Berkley/DATASCI209/Final Project/Data/112/Trajectory/20080124051621.plt"
# df_plt = read_plt(plt_file)

# label_trial = '/content/drive/MyDrive/Berkley/DATASCI209/Final Project/Data/112/labels.txt'
# labels_doc = read_labels(label_trial)

# apply_labels(df_plt, labels_doc)

# folder_trial = '/content/drive/MyDrive/Berkley/DATASCI209/Final Project/Data_Trial/'
# trial_df, trial_results = read_all_users_2(folder_trial)

In [ ]:
folder = '/content/drive/MyDrive/Berkley/DATASCI209/Final Project/Data/'
final_df, final_match = read_all_users_2(folder)

user 161
total points 799
total points not linked to the label 139
percent of points with no label 17.39674593241552
user 104
total points 38572
total points not linked to the label 38250
percent of points with no label 99.16519755262885
user 167
total points 618649
total points not linked to the label 159324
percent of points with no label 25.75353714303264
user 105
total points 1977
total points not linked to the label 142
percent of points with no label 7.182599898836621
user 102
total points 6678
total points not linked to the label 661
percent of points with no label 9.898173105720275
user 085
total points 601871
total points not linked to the label 184463
percent of points with no label 30.648261836838792
user 084
total points 401332
total points not linked to the label 159819
percent of points with no label 39.82214226625337
user 082
total points 172547
total points not linked to the label 120812
percent of points with no label 70.01686497012408
user 062
total points 330302
tota

In [ ]:
final_df.reset_index(inplace=True, drop=True)

In [ ]:
# Change the label to the value from mode_ids
mode_ids = {v: k for k, v in mode_ids.items()}
final_df['label'] = final_df['label'].map(mode_ids)

In [ ]:
# add 8 hours to the time
final_df['time'] = final_df['time'] + datetime.timedelta(hours=8)

# breakout the time into year, day of week, month, and hour
final_df['year'] = final_df['time'].dt.year
final_df['day_of_week'] = final_df['time'].dt.day_name()
final_df['month'] = final_df['time'].dt.month_name()
final_df['hour'] = final_df['time'].dt.hour

In [ ]:
final_df.head()

,lat,lon,alt,time,user,label,segment,start_go_stop,year,day_of_week,month,hour
0,39.976233,116.330567,118.110236,2007-04-12 18:23:15,161,walk,1,start,2007,Thursday,April,18
1,39.975850,116.330400,114.829396,2007-04-12 18:23:25,161,walk,1,go,2007,Thursday,April,18
2,39.975983,116.330467,114.829396,2007-04-12 18:24:37,161,walk,1,go,2007,Thursday,April,18
3,39.976100,116.330500,118.110236,2007-04-12 18:25:03,161,walk,1,go,2007,Thursday,April,18
4,39.976233,116.330567,118.110236,2007-04-12 18:26:25,161,walk,1,stop,2007,Thursday,April,18


# Data Analysis for Data Scoping

Given there are nearly 5M records, we are going to conduct anaysis to identify areas what we can drop data to enhance overall analysis/visualizations.

In [ ]:
# See how much data it outside of the 5th Ring Road of Beijing area
outside_bounds_lat_above = final_df[final_df['lat'] > 49.05].shape[0]
outside_bounds_lat_below = final_df[final_df['lat'] < 39.825].shape[0]
outside_bounds_lon_right = final_df[final_df['lon'] > 116.75].shape[0]
outside_bounds_lon_left = final_df[final_df['lon'] < 116.05].shape[0]

print('Number of records outside of 5th Ring Road Beijing bounds:')
print('lat above', outside_bounds_lat_above)
print('lat below', outside_bounds_lat_below)
print('lon right', outside_bounds_lon_right)
print('lon left', outside_bounds_lon_left)

Number of records outside of 5th Ring Road Beijing bounds:
lat above 20907
lat below 749482
lon right 408280
lon left 628191


In [ ]:
# group by the types of labels
type_grouped = final_df.groupby('label').size().reset_index(name='count')
type_grouped

,label,count
0,airplane,9193
1,bike,948979
2,boat,3566
3,bus,1277291
4,car,511708
5,motorcycle,338
6,run,1975
7,subway,286649
8,taxi,241467
9,train,561031


In [ ]:
# Look at how many rows are in each year
year_grouped = final_df.groupby('year').size().reset_index(name='count')
year_grouped

,year,count
0,2007,42682
1,2008,3437657
2,2009,1025772
3,2010,6360
4,2011,905917


In [ ]:
# Look at the total number of users per year and by type of travel
# final_df_users_type = final_df.groupby(['year', 'user']).size().reset_index(name='count')
# final_df_users_type

final_df_users = final_df.groupby('year')['user'].nunique().reset_index()
final_df_users

,year,user
0,2007,27
1,2008,33
2,2009,6
3,2010,1
4,2011,8


In [ ]:
# Look at the total number of segments per year by label
final_df_segments_type = final_df.groupby(['year', 'label'])['segment'].nunique().reset_index()
# final_df_segments = final_df.groupby('year')['segment'].nunique().reset_index()
final_df_segments_type

,year,label,segment
0,2007,airplane,1
1,2007,bike,78
2,2007,boat,3
3,2007,bus,44
4,2007,car,47
5,2007,subway,7
6,2007,taxi,51
7,2007,train,34
8,2007,walk,127
9,2008,airplane,10


In [ ]:
# List the users that have >50 in the final_match 3 index
users = [i[0] for i in final_match if i[3] <= 50]
users = [int(i) for i in users]

# Group the data by users and count how many unique 'times' and 'segments' there are
final_df_users_segments = final_df[final_df['user'].isin(users)].groupby('user')['segment'].nunique().reset_index().sort_values(by='segment', ascending=False)
final_df_users_times = final_df[final_df['user'].isin(users)].groupby('user')['time'].nunique().reset_index().sort_values(by='time', ascending=False)

# Rename the 'time' column as number of rows
final_df_users_times.rename(columns={'time': 'row_count'}, inplace=True)

print('Total rows for users', sum(final_df_users_times['row_count']))
print('Total segments for users', sum(final_df_users_segments['segment']))

# Merge the two dataframes
final_df_merged = pd.merge(final_df_users_segments, final_df_users_times, on='user')
final_df_merged

Total rows for users 3615673
Total segments for users 6405


,user,segment,row_count
0,85,1097,417407
1,68,942,687220
2,167,818,459184
3,62,550,277038
4,10,432,534136
5,126,428,235437
6,84,419,240859
7,179,310,104821
8,20,188,139513
9,112,140,55369


# Data Trimming

Given the analysis above, we have decided to drop data that is outside of the 5th Ring Road of Beijing.

In [ ]:
# Drop all rows with lat/long outside of the designated boundaries of the 5th Ring Road in Beijing
final_df_trim = final_df[(final_df['lat'] < 49.05) & (final_df['lat'] > 39.825)]
final_df_trim = final_df_trim[(final_df_trim['lon'] < 116.75) & (final_df_trim['lon'] > 116.05)]
final_df_trim.reset_index(inplace=True, drop=True)
final_df_trim

,lat,lon,alt,time,user,label,segment,start_go_stop,year,day_of_week,month,hour
0,39.976233,116.330567,118.110236,2007-04-12 18:23:15,161,walk,1,start,2007,Thursday,April,18
1,39.975850,116.330400,114.829396,2007-04-12 18:23:25,161,walk,1,go,2007,Thursday,April,18
2,39.975983,116.330467,114.829396,2007-04-12 18:24:37,161,walk,1,go,2007,Thursday,April,18
3,39.976100,116.330500,118.110236,2007-04-12 18:25:03,161,walk,1,go,2007,Thursday,April,18
4,39.976233,116.330567,118.110236,2007-04-12 18:26:25,161,walk,1,stop,2007,Thursday,April,18
...,...,...,...,...,...,...,...,...,...,...,...,...
4342314,39.985500,116.183633,1007.217848,2007-10-19 14:53:52,114,walk,6,go,2007,Friday,October,14
4342315,39.985200,116.182983,1040.026247,2007-10-19 14:56:03,114,walk,6,go,2007,Friday,October,14
4342316,39.985067,116.182367,1056.430446,2007-10-19 14:58:30,114,walk,6,go,2007,Friday,October,14
4342317,39.984650,116.182117,1158.136483,2007-10-19 15:01:33,114,walk,6,go,2007,Friday,October,15


In [ ]:
# Only include those rows from year 2009
final_df_trim_2009 = final_df_trim[final_df_trim['year'] == 2009]
final_df_trim_2009.reset_index(inplace=True, drop=True)
final_df_trim_2009

,lat,lon,alt,time,user,label,segment,start_go_stop,year,day_of_week,month,hour
0,39.981115,116.329131,492.0,2009-01-01 11:47:52,85,walk,536,start,2009,Thursday,January,11
1,39.980964,116.329300,492.0,2009-01-01 11:47:54,85,walk,536,go,2009,Thursday,January,11
2,39.980980,116.329273,492.0,2009-01-01 11:47:56,85,walk,536,go,2009,Thursday,January,11
3,39.981084,116.329219,492.0,2009-01-01 11:47:58,85,walk,536,go,2009,Thursday,January,11
4,39.981068,116.329236,492.0,2009-01-01 11:48:00,85,walk,536,go,2009,Thursday,January,11
...,...,...,...,...,...,...,...,...,...,...,...,...
911854,39.995828,116.418254,0.0,2009-09-13 20:51:03,68,bus,968,go,2009,Sunday,September,20
911855,39.995801,116.418256,0.0,2009-09-13 20:51:05,68,bus,968,go,2009,Sunday,September,20
911856,39.995782,116.418267,0.0,2009-09-13 20:51:07,68,bus,968,go,2009,Sunday,September,20
911857,39.995766,116.418277,0.0,2009-09-13 20:51:09,68,bus,968,go,2009,Sunday,September,20


In [ ]:
# Only include those rows from year 2008
final_df_trim_2008 = final_df_trim[final_df_trim['year'] == 2008]
final_df_trim_2008.reset_index(inplace=True, drop=True)
final_df_trim_2008

,lat,lon,alt,time,user,label,segment,start_go_stop,year,day_of_week,month,hour
0,39.966107,116.340690,0.0,2008-01-01 17:42:31,104,bus,14,start,2008,Tuesday,January,17
1,39.966100,116.341192,0.0,2008-01-01 17:42:34,104,bus,14,go,2008,Tuesday,January,17
2,39.966090,116.341647,0.0,2008-01-01 17:42:37,104,bus,14,go,2008,Tuesday,January,17
3,39.966098,116.342063,0.0,2008-01-01 17:42:40,104,bus,14,go,2008,Tuesday,January,17
4,39.966103,116.342487,0.0,2008-01-01 17:42:43,104,bus,14,go,2008,Tuesday,January,17
...,...,...,...,...,...,...,...,...,...,...,...,...
2559881,40.007374,116.322441,129.0,2008-10-12 17:56:23,125,bike,152,go,2008,Sunday,October,17
2559882,40.007401,116.322421,122.0,2008-10-12 17:56:28,125,bike,152,go,2008,Sunday,October,17
2559883,40.007401,116.322421,122.0,2008-10-12 17:56:28,125,bike,152,go,2008,Sunday,October,17
2559884,40.007402,116.322420,121.0,2008-10-12 17:56:33,125,bike,152,go,2008,Sunday,October,17


In [ ]:
# group by the labels of transporation
type_grouped = final_df_trim_2008.groupby('label').size().reset_index(name='count')
type_grouped

,label,count
0,airplane,1806
1,bike,421336
2,boat,1889
3,bus,787740
4,car,281359
5,run,1967
6,subway,95131
7,taxi,129746
8,train,34990
9,walk,803922


In [ ]:
user_grouped = final_df_trim_2008.groupby('year')['user'].nunique().reset_index()
user_grouped

,year,user
0,2008,33


In [ ]:
final_df

,lat,lon,alt,time,user,label,segment,start_go_stop,year,day_of_week,month,hour
0,39.976233,116.330567,118.110236,2007-04-12 18:23:15,161,walk,1,start,2007,Thursday,April,18
1,39.975850,116.330400,114.829396,2007-04-12 18:23:25,161,walk,1,go,2007,Thursday,April,18
2,39.975983,116.330467,114.829396,2007-04-12 18:24:37,161,walk,1,go,2007,Thursday,April,18
3,39.976100,116.330500,118.110236,2007-04-12 18:25:03,161,walk,1,go,2007,Thursday,April,18
4,39.976233,116.330567,118.110236,2007-04-12 18:26:25,161,walk,1,stop,2007,Thursday,April,18
...,...,...,...,...,...,...,...,...,...,...,...,...
5418383,39.985500,116.183633,1007.217848,2007-10-19 14:53:52,114,walk,6,go,2007,Friday,October,14
5418384,39.985200,116.182983,1040.026247,2007-10-19 14:56:03,114,walk,6,go,2007,Friday,October,14
5418385,39.985067,116.182367,1056.430446,2007-10-19 14:58:30,114,walk,6,go,2007,Friday,October,14
5418386,39.984650,116.182117,1158.136483,2007-10-19 15:01:33,114,walk,6,go,2007,Friday,October,15


# Export the Final Dataframe

Export the final dataframe to a CSV for visualizations.

In [ ]:
# Output the final_df to a csv
final_df.to_csv('/content/drive/MyDrive/Berkley/DATASCI209/Final Project/final_df_notrim.csv')

In [ ]:
# Output the updated trimmed df to a csv
# final_df_trim.to_csv('/content/drive/MyDrive/Berkley/DATASCI209/Final Project/final_df_update1.csv')

In [ ]:
# Output the updated trimmed df to a csv
final_df_trim.to_csv('/content/drive/MyDrive/Berkley/DATASCI209/Final Project/final_df_update2.csv')

In [ ]:
# # Output the updated trimmed df to a csv
# final_df_trim_2008.to_csv('/content/drive/MyDrive/Berkley/DATASCI209/Final Project/final_df_2008.csv')

# Exploratory Visualization

In [ ]:
# load my data
df = pd.read_csv('/content/drive/MyDrive/Berkley/DATASCI209/Final Project/final_df_update2.csv')

In [ ]:
# Grouped by hour and the count of rows by labels
df_grouped_hr = df.groupby(['hour','label']).size().reset_index(name='count')
# flatten the df
df_grouped_hr = df_grouped_hr.pivot(index='hour', columns='label', values='count')
#replace all the NaN with 0
df_grouped_hr = df_grouped_hr.fillna(0)

# add a column for total activity recordings that sums up for the entire hour
df_grouped_hr['total'] = df_grouped_hr.sum(axis=1)
df_grouped_hr

label,airplane,bike,bus,car,subway,taxi,train,walk,total
hour,,,,,,,,,
2,0.0,0.0,516.0,0.0,0.0,0.0,0.0,0.0,516.0
3,0.0,0.0,1388.0,0.0,0.0,0.0,0.0,0.0,1388.0
4,0.0,0.0,2846.0,0.0,0.0,0.0,0.0,0.0,2846.0
5,0.0,280.0,1759.0,0.0,0.0,0.0,0.0,606.0,2645.0
6,0.0,844.0,6888.0,0.0,12.0,0.0,0.0,1068.0,8812.0
7,0.0,12080.0,17053.0,478.0,2646.0,0.0,53.0,4063.0,36373.0
8,0.0,27704.0,11052.0,3541.0,24541.0,2630.0,0.0,6983.0,76451.0
9,0.0,7513.0,15039.0,13222.0,33153.0,4729.0,80.0,8331.0,82067.0
10,0.0,9654.0,19911.0,9508.0,4039.0,3672.0,45.0,10402.0,57231.0


In [ ]:
# group the dataframe by user and then segment keeping the time for start and stop in the start_go_stop column and the label
pivot_df = df.groupby(['user','segment', 'label']).agg({'time': ['min', 'max']}).reset_index()

# change the min and max column to datetime datatypes
pivot_df['time'] = pivot_df['time'].apply(pd.to_datetime)

In [ ]:
# calculate the minutes between min and max
pivot_df['minutes'] = (pivot_df['time']['max'] - pivot_df['time']['min']).dt.total_seconds() / 60
pivot_df

user segment   label                time                        minutes
                                          min                 max           
0      10       1   train 2008-03-28 22:54:40 2008-03-28 23:04:06   9.433333
1      10      34   train 2008-04-06 17:04:31 2008-04-06 17:18:36  14.083333
2      10      35     bus 2008-04-06 17:30:20 2008-04-06 18:21:26  51.100000
3      10      53     bus 2008-06-17 17:44:44 2008-06-17 18:16:27  31.716667
4      10      54    walk 2008-06-17 18:53:07 2008-06-17 19:00:07   7.000000
...   ...     ...     ...                 ...                 ...        ...
8784  179     314     bus 2008-11-17 14:59:58 2008-11-17 15:06:14   6.266667
8785  179     315    walk 2008-11-17 15:06:16 2008-11-17 15:14:32   8.266667
8786  179     316     bus 2008-11-29 09:58:05 2008-11-29 10:01:37   3.533333
8787  179     317    walk 2008-11-29 10:01:39 2008-11-29 10:05:43   4.066667
8788  179     318  subway 2008-11-29 10:08:01 2008-11-29 10:43:37  35.600000

[8789 rows x 6 columns]

In [ ]:
# create a new column from the max time column with the hour
pivot_df.columns = ['_'.join(col).rstrip('_') for col in pivot_df.columns.values]

pivot_df['hour'] = pivot_df['time_max'].dt.hour

# create a new column from the time column with the year
pivot_df['year'] = pivot_df['time_max'].dt.year

# create a new column from the time column with the month
pivot_df['month'] = pivot_df['time_max'].dt.month_name()

# create a new column from the time column with the day of week
pivot_df['day_of_week'] = pivot_df['time_max'].dt.day_name()

pivot_df

,user,segment,label,time_min,time_max,minutes,hour,year,month,day_of_week
0,10,1,train,2008-03-28 22:54:40,2008-03-28 23:04:06,9.433333,23,2008,March,Friday
1,10,34,train,2008-04-06 17:04:31,2008-04-06 17:18:36,14.083333,17,2008,April,Sunday
2,10,35,bus,2008-04-06 17:30:20,2008-04-06 18:21:26,51.100000,18,2008,April,Sunday
3,10,53,bus,2008-06-17 17:44:44,2008-06-17 18:16:27,31.716667,18,2008,June,Tuesday
4,10,54,walk,2008-06-17 18:53:07,2008-06-17 19:00:07,7.000000,19,2008,June,Tuesday
...,...,...,...,...,...,...,...,...,...,...
8784,179,314,bus,2008-11-17 14:59:58,2008-11-17 15:06:14,6.266667,15,2008,November,Monday
8785,179,315,walk,2008-11-17 15:06:16,2008-11-17 15:14:32,8.266667,15,2008,November,Monday
8786,179,316,bus,2008-11-29 09:58:05,2008-11-29 10:01:37,3.533333,10,2008,November,Saturday
8787,179,317,walk,2008-11-29 10:01:39,2008-11-29 10:05:43,4.066667,10,2008,November,Saturday


In [ ]:
pivot_df_hour = pivot_df.copy()

In [ ]:
# drop the time and time_end columns
pivot_df.drop(columns=['time_min', 'time_max'], inplace=True)

In [ ]:
pivot_df.head()

,user,segment,label,minutes,hour,year,month,day_of_week
0,10,1,train,9.433333,23,2008,March,Friday
1,10,34,train,14.083333,17,2008,April,Sunday
2,10,35,bus,51.100000,18,2008,April,Sunday
3,10,53,bus,31.716667,18,2008,June,Tuesday
4,10,54,walk,7.000000,19,2008,June,Tuesday


In [ ]:
# export the csv
# pivot_df.to_csv('/content/drive/MyDrive/Berkley/DATASCI209/Final Project/pivot_df.csv')

In [ ]:
# create columns for the hours in the day, 0 to 23
# if the hour is greater than the hour in time_min but less than the hour in time_max make the value true

for i in range(24):
  pivot_df_hour[f'hour_{i}'] = np.where((pivot_df_hour['time_min'].dt.hour <= i) & (pivot_df_hour['time_max'].dt.hour >= i), 1, 0)
  pivot_df_hour[f'hour_{i}'] = pivot_df_hour[f'hour_{i}'].fillna(0)
  pivot_df_hour[f'hour_{i}'] = pivot_df_hour[f'hour_{i}'].astype(int)

# drop the hour column
pivot_df_hour.drop(columns=['hour', 'time_max', 'time_min'], inplace=True)

pivot_df_hour.head()


,user,segment,label,minutes,year,month,day_of_week,hour_0,hour_1,hour_2,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,10,1,train,9.433333,2008,March,Friday,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,10,34,train,14.083333,2008,April,Sunday,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,10,35,bus,51.100000,2008,April,Sunday,0,0,0,...,0,0,0,1,1,0,0,0,0,0
3,10,53,bus,31.716667,2008,June,Tuesday,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,10,54,walk,7.000000,2008,June,Tuesday,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [ ]:
# create a pivot chart with hours on the left and counts of '1'
counts = pivot_df_hour.groupby('label')[['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4',
                        'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9',
                        'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14',
                        'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19',
                        'hour_20', 'hour_21', 'hour_22', 'hour_23']].sum().transpose()

counts['total'] = counts.sum(axis=1)
counts

label,airplane,bike,boat,bus,car,run,subway,taxi,train,walk,total
hour_0,0,39,0,1,5,0,0,8,0,52,105
hour_1,0,25,0,1,2,0,0,3,0,22,53
hour_2,0,12,0,1,1,0,0,2,0,9,25
hour_3,0,2,0,2,1,0,0,1,0,6,12
hour_4,0,4,0,5,1,0,0,2,0,4,16
hour_5,0,2,0,12,1,0,1,2,0,6,24
hour_6,0,7,0,53,15,0,2,1,2,27,107
hour_7,1,72,0,140,55,0,26,17,20,139,470
hour_8,4,178,0,174,81,0,104,43,11,348,943
hour_9,0,216,0,174,138,0,161,33,17,477,1216


In [ ]:
# export the csv
counts.to_csv('/content/drive/MyDrive/Berkley/DATASCI209/Final Project/hour_segment.csv')

In [ ]:
# calculate the average # of segments for each hour for each label
counts['avg'] = counts['total'] / 24
counts

label,airplane,bike,boat,bus,car,run,subway,taxi,train,walk,total,avg_segments,avg
hour_0,0,39,0,1,5,0,0,8,0,52,105,4.375000,4.375000
hour_1,0,25,0,1,2,0,0,3,0,22,53,2.208333,2.208333
hour_2,0,12,0,1,1,0,0,2,0,9,25,1.041667,1.041667
hour_3,0,2,0,2,1,0,0,1,0,6,12,0.500000,0.500000
hour_4,0,4,0,5,1,0,0,2,0,4,16,0.666667,0.666667
hour_5,0,2,0,12,1,0,1,2,0,6,24,1.000000,1.000000
hour_6,0,7,0,53,15,0,2,1,2,27,107,4.458333,4.458333
hour_7,1,72,0,140,55,0,26,17,20,139,470,19.583333,19.583333
hour_8,4,178,0,174,81,0,104,43,11,348,943,39.291667,39.291667
hour_9,0,216,0,174,138,0,161,33,17,477,1216,50.666667,50.666667
